# Compute building energy demand

This notebook loads data about a list of building and compute an estimation of their their energy demand based on construction periods.

## Import and install the necessary modules

In [14]:
import pandas as pd
import numpy as np
import sys

!pip install osem
from osem.building_demand import SpecificEnergyRequirements


## Import the data from an excel file

In [2]:
filename = r'data_example/VillageChampignac.xlsx'
data_building = pd.read_excel(filename, index_col=0)
# print(data_building)

## Pass from year to period code

In [8]:
def get_period_from_year(year):
    """
 This function get the code of the construciton period from the year of construction  
 :param year: the construction year (int)  
 :return: the code construction from RegBL
 """ 
    years_start = np.array([1,1919,1946,1961,1971,1981,1986,1991,1996,2001,2006,2011,2015])
    codes = range(8011,8024)
    ind = np.searchsorted(years_start, year)
    return codes[ind-1]

## Get energy demand by building

In [23]:
# preparation
demand_all_building = pd.DataFrame(data=0, columns=['heat', 'total'], index=data_building.index)

# take one building after the other and calculate heat/warm water demand
for ind, building_here in data_building.iterrows():
    period_here = get_period_from_year(building_here[1])
    args = {'affectation': 2, 'period':period_here}  # individual house -> affection=2
    res = SpecificEnergyRequirements(args).calculate() 
    demand_all_building.loc[ind, : ] = np.array([res["hS"], res["hS"]+res["hW"]]) * building_here['SRE [m2]']
demand_all_building = pd.concat((data_building, demand_all_building), sort=True, axis=1)

## Save data into a file

In [24]:
demand_all_building.to_excel('data_example/DemandChampignac.xlsx')